## NVIDIA NEMO TRAINING: STAGE1
* In this stage we are going to drop all rows that have missing values in the age and gender columns so that we can have a generalizable model to a diverse set of gender and age
* Hence we get approximately 40k trainining data vs 10k validation data

In [ ]:

!pip install nemo_toolkit['all']

#### Login to your hugging face
* If you want to save the finetuned model you have to be logged in to your hugging face account
* So provide your token in the resulting result

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd
import tarfile
import os
import json
import librosa
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl


from omegaconf import DictConfig
import copy

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split


#### Load the files
* I used kaggle as my training kernel
* Hence all these paths are relative to kaggle
* If you are running this on collab you have to take care of the paths

In [ ]:

train = pd.read_csv("/kaggle/input/mozilla-swahili/train.tsv",sep = '\t' )
test = pd.read_csv("/kaggle/input/mozilla-swahili/SampleSubmission(6).csv")
train.dropna(subset = ['age', 'gender'], inplace = True)
train = train[~train['path'].isin(["common_voice_sw_30035155.mp3", "common_voice_sw_30377114.mp3","common_voice_sw_30318282.mp3"])]

train['gender_age'] = train['gender'] + "_" + train['age']
train['sentence'] = train['sentence'].str.lower()


df_train, df_valid = train_test_split(train, test_size = 0.2, stratify = train['gender_age'], shuffle =True, random_state = 42)
df_valid = df_valid[~df_valid['path'].isin(['common_voice_sw_31818110.mp3'])]

df_train['path'] = "/kaggle/input/mozilla-swahili/train0-002/train/" + df_train['path']
df_valid['path'] = "/kaggle/input/mozilla-swahili/train0-002/train/" + df_valid['path']
test['path'] = "/kaggle/input/mozilla-swahili/Mozilla Foundation - Mozilla Common Voice Hackathon-20231013T080731Z-001/Mozilla Foundation - Mozilla Common Voice Hackathon/test0/" + test['path']
test["sentence"] = ""

# Save the train and validation DataFrames to CSV files
df_train.to_csv('train_split.csv', index=False)
df_valid.to_csv('valid_split.csv', index=False)
test.to_csv("test.csv", index=False)


display(df_train.head(), df_train.shape, df_valid.head(), df_valid.shape)









### Load the base Model that I am going to finetune
* I am gonoing to finetune this model

In [ ]:

asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="yonas/stt_rw_sw_lg_conformer_ctc_large")

#### Build a Manifest  for both train and valid set to feed it to the nemo model

In [ ]:
display(df_train.head(), df_valid.head())

In [ ]:
df_train.shape, df_valid.shape

In [ ]:
# Function to build a manifest
def build_manifest(dataframe, manifest_path):
    df = dataframe.copy()
    with open(manifest_path, 'w') as fout:
        for _, row in df.iterrows():
            transcript = row['sentence'].lower().strip()
            file_id = row['path']
            duration = librosa.get_duration(path= file_id)

            # Write the metadata to the manifest
            metadata = {
                "audio_filepath": file_id,
                "duration": duration,
                "text": transcript
            }
            json.dump(metadata, fout)
            fout.write('\n')


In [ ]:
train_manifest = 'train_manifest.json'
if not os.path.isfile(train_manifest):
    build_manifest(df_train, train_manifest)
    print("Training manifest created.")

In [ ]:


valid_manifest = 'valid_manifest.json'
if not os.path.isfile(valid_manifest):
    build_manifest(df_valid, valid_manifest)
    print("Validation manifest created.")



#### Load Configurations

In [ ]:
# --- Config Information ---#

try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel.yaml import YAML
config_path = './configs/config.yaml'

if not os.path.exists(config_path):
    # Grab the config we'll use in this example
    BRANCH = 'main'
    !mkdir configs
    !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

#### Modify Configurations

In [ ]:
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = valid_manifest
# # Bigger batch-size = bigger throughput
params['model']['train_ds']['batch_size'] = 12
params['model']['validation_ds']['batch_size'] = 12

print(params['model']['optim'])  ###


# import copy
new_opt = copy.deepcopy(params['model']['optim'])
new_opt['lr'] = 1e-03

print('new_opt', new_opt)


#### Setup the Model with the new config

In [ ]:
# Use the smaller learning rate we set before  ####
asr_model.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the data we'll use for fine-tuning as the training set
asr_model.setup_training_data(train_data_config=params['model']['train_ds'])

# Point to the new validation data for fine-tuning
asr_model.setup_validation_data(val_data_config=params['model']['validation_ds'])


#### Training: woohooo

In [ ]:
%%time
n_epochs = 15
trainer = pl.Trainer(max_epochs=n_epochs)
trainer.fit(asr_model)

#### Save The Model and Restore It


In [ ]:
#save the Model
asr_model.save_to(save_path='/kaggle/working/nemo_40k_10k_1e-03_15epochs_upd.nemo')


#### Push the Model to Hugging Face

In [ ]:
!git config --global credential.helper store

In [ ]:
model_name = "stt_sw_40k_10k_1e_03_15_epoch_update"

In [ ]:
#Restore
model = nemo_asr.models.EncDecCTCModel.restore_from('/kaggle/working/nemo_40k_10k_1e-03_15epochs_upd.nemo')

#### Create a Hugging Face Model
* If you wanna save your model to hugging face, make sure you provided your token in the top level hugging face code block, else it will fail

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
username = api.whoami()['name']

#### Add readme and push your model to hugging face

In [ ]:
try:
  api.create_repo(repo_id=model_name)
  print("Successfully created repository !")
except Exception as e:
  print("Repository is possibly already created. Refer to error here - \n\n", e)

In [ ]:
from huggingface_hub import Repository
local_dir = f'model-{model_name}/'
hf_model_name = f'{username}/{model_name}'

commit_message = "Upload model"
model_filename = f'{model_name}.nemo'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  model.save_to(model_filename)

print("Finished uploading model to :", hf_model_name)

#### Model Card

In [ ]:
hf_url = f'https://huggingface.co/{username}/{model_name}'
print(f"Visit {hf_url} to manually edit your model card")

In [ ]:
TAGS = [
    "automatic-speech-recognition",  # Task id, refer to https://github.com/huggingface/datasets/blob/master/src/datasets/utils/resources/tasks.json for allowed values.
    "speech",  # add as many other tags as required
    "audio",
    "CTC",
    "Conformer",
    "Transformer",
    "NeMo",  # required for library identification
    "pytorch",  # required, for toolkit identification
    # "hf-asr-leaderboard",  # Should only be used if model is evaluated on benchmark scores for ASR.
]

In [ ]:
# Replace all spaces with `-`
DATASETS = [
    "librispeech_asr",
    "mozilla-foundation/common_voice_custom_40k_10k",
    "Multilingual-LibriSpeech-(>40-hours)",
]

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any

@dataclass
class NeMoHuggingFaceModelConfig:
  language: List[str]
  license: str

  library_name: str = "nemo"
  datasets: List[str] = field(default_factory=lambda: DATASETS)
  thumbnail: Optional[str] = None
  tags: List[str] = field(default_factory=lambda: TAGS)
  model_index: Any = field(default_factory=lambda: [dict(name=model_name, results=[])])

In [ ]:
from omegaconf import DictConfig, OmegaConf, open_dict
config = NeMoHuggingFaceModelConfig(language=['en'], license="cc-by-4.0")  # choose appropriate license here
config = OmegaConf.structured(config)

with open_dict(config):
  # Update `model_index` to `model-index`
  model_index = config.pop('model_index')
  config['model-index'] = model_index

  # Replace all spaces with `-` in datasets
  normalized_datasets = [ds_name.replace(" ", "-") for ds_name in config['datasets']]
  config['datasets'] = OmegaConf.create(normalized_datasets)

print(OmegaConf.to_yaml(config))

In [ ]:
hf_model_name = f'{username}/{model_name}'

TEMPLATE = f"""
## Model Overview

<DESCRIBE IN ONE LINE THE MODEL AND ITS USE>

## NVIDIA NeMo: Training

To train, fine-tune or play with the model you will need to install [NVIDIA NeMo](https://github.com/NVIDIA/NeMo). We recommend you install it after you've installed latest Pytorch version.
```
pip install nemo_toolkit['all']
```

## How to Use this Model

The model is available for use in the NeMo toolkit [3], and can be used as a pre-trained checkpoint for inference or for fine-tuning on another dataset.

### Automatically instantiate the model

```python
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("{hf_model_name}")
```

### Transcribing using Python
First, let's get a sample
```
wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
```
Then simply do:
```
asr_model.transcribe(['2086-149220-0033.wav'])
```

### Transcribing many audio files

```shell
python [NEMO_GIT_FOLDER]/examples/asr/transcribe_speech.py \
 pretrained_name="{hf_model_name}" \
 audio_dir="<DIRECTORY CONTAINING AUDIO FILES>"
```

### Input

This model accepts 16000 KHz Mono-channel Audio (wav files) as input.

### Output

This model provides transcribed speech as a string for a given audio sample.

## Model Architecture

<ADD SOME INFORMATION ABOUT THE ARCHITECTURE>

## Training

<ADD INFORMATION ABOUT HOW THE MODEL WAS TRAINED - HOW MANY EPOCHS, AMOUNT OF COMPUTE ETC>

### Datasets

<LIST THE NAME AND SPLITS OF DATASETS USED TO TRAIN THIS MODEL (ALONG WITH LANGUAGE AND ANY ADDITIONAL INFORMATION)>

## Performance

<LIST THE SCORES OF THE MODEL -
      OR
USE THE Hugging Face Evaluate LiBRARY TO UPLOAD METRICS>

## Limitations

<DECLARE ANY POTENTIAL LIMITATIONS OF THE MODEL>

Eg:
Since this model was trained on publicly available speech datasets, the performance of this model might degrade for speech which includes technical terms, or vernacular that the model has not been trained on. The model might also perform worse for accented speech.


## References

<ADD ANY REFERENCES HERE AS NEEDED>

[1] [NVIDIA NeMo Toolkit](https://github.com/NVIDIA/NeMo)

"""

In [ ]:
local_dir = f'model-{model_name}/'
hf_model_name = f'{username}/{model_name}'

commit_message = "Upload config"
filename = 'readme_template.md'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  with open(filename, 'w') as f:
    f.write("---\n")
    f.write(OmegaConf.to_yaml(config))
    f.write("\n---\n\n")
    f.write(TEMPLATE)


In [ ]:
hf_url = f'https://huggingface.co/{username}/{model_name}'
print(f"Visit {hf_url} to edit your model card from the generated template file `{filename}`")